In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model

def load():
    (trainX, _), (_, _) = tf.keras.datasets.mnist.load_data()
    X = trainX.astype('float32') / 127.5 - 1
    X = np.expand_dims(X, axis=-1)
    return X

def discriminator():
    input_layer = layers.Input(shape=(28, 28, 1))
    x = layers.Flatten()(input_layer)
    x = layers.Dense(128)(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.4)(x)  
    output_layer = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003, beta_1=0.5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

def generator(latent_dim):
    input_layer = layers.Input(shape=(latent_dim,))
    x = layers.Dense(128, activation='relu')(input_layer)
    x = layers.Dense(784, activation='tanh')(x)
    output_layer = layers.Reshape((28, 28, 1))(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

def real(dataset, n_samples):
    idx = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[idx]
    y = np.ones((n_samples, 1)) * 0.9  
    return X, y

def fake(generator, latent_dim, n_samples):
    noise = np.random.randn(n_samples, latent_dim)
    X = generator.predict(noise, verbose=0)
    y = np.zeros((n_samples, 1))
    return X, y

def train_discriminator(discriminator, generator, dataset, latent_dim, epochs=100, batch_size=128):
    batches_per_epoch = dataset.shape[0] // batch_size
    for epoch in range(epochs):
        for _ in range(batches_per_epoch):
            # Train on real data
            X_real, y_real = real(dataset, batch_size // 2)
            X_fake, y_fake = fake(generator, latent_dim, batch_size // 2)
            
          
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            
           
            discriminator_loss, discriminator_acc = discriminator.train_on_batch(X, y)
        
        print(f"Epoch {epoch + 1}/{epochs}, Discriminator Loss: {discriminator_loss:.3f}, Accuracy: {discriminator_acc:.3f}")
def train_gan(generator, discriminator, dataset, latent_dim, epochs=100, batch_size=128):
    for epoch in range(epochs):
        noise = np.random.randn(batch_size, latent_dim)
        X_fake = generator.predict(noise, verbose=0)
        y_fake = np.zeros((batch_size, 1))

      
        discriminator.trainable = False  
        generator_loss = discriminator.train_on_batch(X_fake, y_fake)
        
      
        discriminator.trainable = True 
        X_real, y_real = real(dataset, batch_size // 2)
        X_fake, y_fake = fake(generator, latent_dim, batch_size // 2)
        
     
        X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
        
 
        discriminator_loss, discriminator_acc = discriminator.train_on_batch(X, y)

   
        print(f"Epoch {epoch + 1}/{epochs}, Generator Loss: {generator_loss[0]:.3f}, Discriminator Loss: {discriminator_loss:.3f}, Accuracy: {discriminator_acc:.3f}")

if __name__ == "__main__":
    latent_dim = 100
    dataset = load()
    discriminator = discriminator()
    generator = generator(latent_dim)

    print("Discriminator Summary:")
    discriminator.summary()

    print("Generator Summary:")
    generator.summary()

    train_gan(generator, discriminator, dataset, latent_dim, epochs=100) 

Discriminator Summary:


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_10 (LeakyReLU)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,609 (393.00 KB)

 Trainable params: 100,609 (393.00 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 784)            │       101,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_6 (Reshape)             │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 114,064 (445.56 KB)

 Trainable params: 114,064 (445.56 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100, Generator Loss: 0.628, Discriminator Loss: 0.823, Accuracy: 0.504
Epoch 2/100, Generator Loss: 0.758, Discriminator Loss: 0.826, Accuracy: 0.500
Epoch 3/100, Generator Loss: 0.789, Discriminator Loss: 0.834, Accuracy: 0.488
Epoch 4/100, Generator Loss: 0.806, Discriminator Loss: 0.829, Accuracy: 0.492
Epoch 5/100, Generator Loss: 0.809, Discriminator Loss: 0.835, Accuracy: 0.475
Epoch 6/100, Generator Loss: 0.819, Discriminator Loss: 0.840, Accuracy: 0.477
Epoch 7/100, Generator Loss: 0.824, Discriminator Loss: 0.840, Accuracy: 0.478
Epoch 8/100, Generator Loss: 0.826, Discriminator Loss: 0.843, Accuracy: 0.476
Epoch 9/100, Generator Loss: 0.829, Discriminator Loss: 0.844, Accuracy: 0.472
Epoch 10/100, Generator Loss: 0.833, Discriminator Loss: 0.842, Accuracy: 0.476
Epoch 11/100, Generator Loss: 0.832, Discriminator Loss: 0.838, Accuracy: 0.477
Epoch 12/100, Generator Loss: 0.828, Discriminator Loss: 0.837, Accuracy: 0.483
Epoch 13/100, Generator Loss: 0.828, Discriminato